# More Voting Mechanism Simulation Examples

Our goal here is to show basic use cases for the code, and do basic proof-of-concept on possible future extensions.  

In this notebook, we look at which helper mechanisms may be necessary to extend a voting design. 

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import os
from random import choice
from typing import Dict

# Custom imports
from custom_types import UserNFTs
from helper import calc_voter_weights_from_NFT_weight
from single_choice_weighted_plurality import SingleChoiceWeightedPlurality

## Inputting Sample Data as a Dictionary

In the cell below, we load a data file of which users currently hold which TE Academy NFTs. 

In [2]:
VOTER_DATA_FILENAME = "data/nft_data_may_22_cleaned.csv"

voter_data = pd.read_csv(VOTER_DATA_FILENAME)

# We may need to drop a specific column. 
voter_data.drop(columns = ['Unnamed: 0'], 
                inplace = True)
voter_data.set_index('ID', inplace = True)

Now we convert the data to a dictionary. 

In [3]:
sample_voters = voter_data.to_dict(orient='index')


Let's see what this new structure looks like. 

In [4]:
sample_voters

{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 0,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 0,
  'SPEAKER_BARCAMP_PARIS_23': 0,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 0,
  'FUND_MOD1': 0,
  'FUND_MOD2': 0,
  'FUND_MOD3': 1,
  'FUND_MOD4': 1,
  'FUND_MOD5': 0},
 '0x049debbad61a20e21e872b06ef4f25be1253c802': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 1,
  'FUND_AUTHOR': 0,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 0,
  'SPEAKER_BARCAMP_PARIS_23': 0,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 0,
  'FUND_MOD1': 0,
  'FUND_MOD2': 0,
  'FUND_MOD3': 1,
  'FUND_MOD4': 1,
  'FUND_MOD5': 1},
 '0xa481db1ac55683dfb2847c02e417b7e411bcbbea': {'NFTREP_V1': 1,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 0,
  'STUDY_GR

In [5]:
voters = {key: UserNFTs(sample_voters.get(key))
          for key, _ in sample_voters.items()
          }

In [6]:
voters

{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 0,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 0,
  'SPEAKER_BARCAMP_PARIS_23': 0,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 0,
  'FUND_MOD1': 0,
  'FUND_MOD2': 0,
  'FUND_MOD3': 1,
  'FUND_MOD4': 1,
  'FUND_MOD5': 0},
 '0x049debbad61a20e21e872b06ef4f25be1253c802': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 1,
  'FUND_AUTHOR': 0,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 0,
  'SPEAKER_BARCAMP_PARIS_23': 0,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 0,
  'FUND_MOD1': 0,
  'FUND_MOD2': 0,
  'FUND_MOD3': 1,
  'FUND_MOD4': 1,
  'FUND_MOD5': 1},
 '0xa481db1ac55683dfb2847c02e417b7e411bcbbea': {'NFTREP_V1': 1,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 0,
  'STUDY_GR

In [7]:
# Get the first voter
first_voter = list(sample_voters.values())[0]

# For the moment, assume all NFT weights are equal. 
NFT_weights = {key: 1.0 
               for key
               in first_voter.keys()
               }

Now we use the `calc_voter_weights_from_NFT_weights` function from `helper.py` to calculate what voter weights would be for this scenario. 

In [8]:
voter_weights = calc_voter_weights_from_NFT_weight(voters,
                                                   NFT_weights)

In [9]:
voter_weights

{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87': {'weight': 2.0},
 '0x049debbad61a20e21e872b06ef4f25be1253c802': {'weight': 4.0},
 '0xa481db1ac55683dfb2847c02e417b7e411bcbbea': {'weight': 4.0},
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370': {'weight': 6.0},
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242': {'weight': 6.0},
 '0xfb2069bf55eb844694d8028df74dafe550821ffc': {'weight': 5.0},
 '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f': {'weight': 6.0},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'weight': 9.0},
 '0x6697aeedc3eadbcfb25962c2045aedf438b4c1f2': {'weight': 4.0},
 '0x6a7becf86b56fb68c948cb0a9fe9961370079190': {'weight': 5.0},
 '0xc8eba9130f40a0ea9be530407e5b4f7199118864': {'weight': 6.0},
 '0x6e3556f2c8f29559ed67642a1813b177f1871613': {'weight': 6.0},
 '0x278a4bd5a8d09bcf11b1052ec3129a7efa75d213': {'weight': 6.0},
 '0x28ddb1f987f1d433eb82eef644b006741977f456': {'weight': 6.0},
 '0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'weight': 8.0},
 '0xa587b5aa3ddf49559cde74140456bf65d716

For comparison's sake, we will also look at giving each wallet a uniform weight of 1.0. This is the same as 1-wallet-1-vote. 

In [10]:
uniform_weights = {key: {"weight": 1.0} for key in sample_voters.keys()}

In [11]:
uniform_weights

{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87': {'weight': 1.0},
 '0x049debbad61a20e21e872b06ef4f25be1253c802': {'weight': 1.0},
 '0xa481db1ac55683dfb2847c02e417b7e411bcbbea': {'weight': 1.0},
 '0x024bd3d1c3b6ba2277a744d7a99fbade9404a370': {'weight': 1.0},
 '0xa0b026b4b9e19ff2cbf5afbd9a679d93dedc2242': {'weight': 1.0},
 '0xfb2069bf55eb844694d8028df74dafe550821ffc': {'weight': 1.0},
 '0x222cf246a3d56d61030ca49f5e6ffd6c20c3dc4f': {'weight': 1.0},
 '0x5658f4b362e95c0bc32d5ce0051457882db92ee5': {'weight': 1.0},
 '0x6697aeedc3eadbcfb25962c2045aedf438b4c1f2': {'weight': 1.0},
 '0x6a7becf86b56fb68c948cb0a9fe9961370079190': {'weight': 1.0},
 '0xc8eba9130f40a0ea9be530407e5b4f7199118864': {'weight': 1.0},
 '0x6e3556f2c8f29559ed67642a1813b177f1871613': {'weight': 1.0},
 '0x278a4bd5a8d09bcf11b1052ec3129a7efa75d213': {'weight': 1.0},
 '0x28ddb1f987f1d433eb82eef644b006741977f456': {'weight': 1.0},
 '0xfcbc07905fee2d64025461b8ddb27f77f256827f': {'weight': 1.0},
 '0xa587b5aa3ddf49559cde74140456bf65d716

Now let's run 100000 trials where we randomly assign voter preference, and calculate what the result would be under each type of weight. 

In [12]:
SCWPCalculator = SingleChoiceWeightedPlurality()

NUM_EXPERIMENTS = 100_000

# Create an empty DataFrame to store simulation results
results_list = [] 

for k in range(NUM_EXPERIMENTS):
    sample_choices = {
                   key: choice(["A","B","C","D"])
                   for key in sample_voters.keys()
                 }
    weighted_winner, weighted_results  = SCWPCalculator.calculate(voter_weights,
                            sample_choices)
    uniform_winner, uniform_results = SCWPCalculator.calculate(uniform_weights,
                            sample_choices)
    # Create an empty DataFrame to store simulation results
    results_list.append({'Experiment': k,
                        'weighted_winner': weighted_winner, 
                        'weighted_candidate_scores': weighted_results,
                        'uniform_winner': uniform_winner,
                        'uniform_candidate_scores': uniform_results}
                        )


results_df = pd.DataFrame(results_list)

#TODO: Refactor for speed if needed.  

    

The data below is produced by the above code. 

In [13]:
results_df

,Experiment,weighted_winner,weighted_candidate_scores,uniform_winner,uniform_candidate_scores
0,0,D,"{'C': 195.0, 'D': 234.0, 'A': 213.0, 'B': 219.0}",D,"{'C': 80.0, 'D': 95.0, 'A': 83.0, 'B': 89.0}"
1,1,A,"{'C': 201.0, 'D': 223.0, 'B': 209.0, 'A': 228.0}",A,"{'C': 79.0, 'D': 90.0, 'B': 85.0, 'A': 93.0}"
2,2,D,"{'B': 186.0, 'C': 203.0, 'A': 216.0, 'D': 256.0}",D,"{'B': 89.0, 'C': 85.0, 'A': 76.0, 'D': 97.0}"
3,3,C,"{'A': 191.0, 'C': 253.0, 'B': 235.0, 'D': 182.0}",B,"{'A': 88.0, 'C': 89.0, 'B': 91.0, 'D': 79.0}"
4,4,D,"{'D': 236.0, 'C': 210.0, 'B': 214.0, 'A': 201.0}",D,"{'D': 100.0, 'C': 86.0, 'B': 87.0, 'A': 74.0}"
...,...,...,...,...,...
99995,99995,D,"{'A': 190.0, 'D': 238.0, 'C': 224.0, 'B': 209.0}",D,"{'A': 66.0, 'D': 106.0, 'C': 92.0, 'B': 83.0}"
99996,99996,A,"{'C': 215.0, 'A': 232.0, 'B': 200.0, 'D': 214.0}",D,"{'C': 87.0, 'A': 87.0, 'B': 75.0, 'D': 98.0}"
99997,99997,C,"{'C': 262.0, 'D': 205.0, 'A': 218.0, 'B': 176.0}",C,"{'C': 92.0, 'D': 86.0, 'A': 89.0, 'B': 80.0}"
99998,99998,C,"{'B': 211.0, 'A': 194.0, 'C': 235.0, 'D': 221.0}",C,"{'B': 80.0, 'A': 82.0, 'C': 98.0, 'D': 87.0}"


Below we check: "Which proportion of the time did the two different weightings produce the same result?"

In [14]:
(results_df['weighted_winner'] == results_df['uniform_winner']).mean()

0.67398